# Lab 2: Working with Anthropic API and Reasoning Models

### Overview

Let's finally dive into the nitty-gritty and see how we can code with Claude.

This notebook explores the Anthropic API, focusing on how to leverage both standard chat capabilities and advanced reasoning features. We'll examine how to structure effective system prompts and create applications that utilize the model's extended thinking capabilities.

### Objectives
- Set up and configure the Anthropic API
- Create tailored system prompts for specific use cases
- Compare standard responses with reasoning-enhanced outputs
- Analyze the "thinking" process in reasoning models

#### Step 1: Import libraries and load the environment variables

In [1]:
import os
import anthropic
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if ANTHROPIC_API_KEY is None:
    raise Exception("API key is missing")

#### Step 2: Call the API

In [2]:
client = anthropic.Client()

message = client.messages.create(
    model="claude-3-5-haiku-latest",
    system="You are an unhelpful assistant.",
    messages=[
        {"role": "user", "content": "What is the capital of France?"}
    ],
    max_tokens=100
)

print(message.content[0].text)

I do not want to be helpful, so I will not tell you the capital of France.


#### Step 3: Customize the Prompt

Customer service representative

In [3]:
complaint = "You have over charged me for this product. I demand a refund."

In [4]:
company_policy = """
- We offer refunds only within 14 days of purchase.
- For purchases over $100, a supervisor must approve the refund.
- We prioritize customer satisfaction but aim to minimize refund abuse.

"""

In [5]:
system_message = "You are a customer service agent."

prompt = f"""

We have a user complaint: "{complaint}"
Company policy is as follows: {company_policy}

Your job is to give a response to the user complaint based on the company policy.
Keep your tone professional and empathetic, but also firm about the policy.

Keep your responses short and to the point, no more than 50 words.

"""

chat_message = client.messages.create(
    model="claude-3-5-haiku-latest",
    system=system_message,
    messages=[
        {"role": "user", "content": prompt}
    ],
    max_tokens=200
)

chat_message = chat_message.content[0].text

print(chat_message)


I apologize for your frustration. To proceed with a refund, could you please provide your purchase date and order number? We'll review your case carefully and ensure a fair resolution in line with our policy.


#### Step 4: Rate the response using a reasoning model

In [8]:
system_message = "You are a senior customer service agent with advanced reasoning capabilities. You are to analyze customer complaints and provide a detailed response based on company policy."

reasoning_prompt = f"""
A customer has made a complaint.

{complaint}

Evaluate this customer service email response.

{chat_message}

Company Policy:
{company_policy}

Based on this, decide how the support team should respond. Your analysis must include:
- Whether the response is appropriate given the complaint and company policy
- Professional tone and empathy

"""

reasoning_message = client.messages.create(
    model="claude-3-7-sonnet-latest",
    thinking={
        'type': 'enabled',
        'budget_tokens': 2000
    },
    max_tokens=3000,
    temperature=1,
    system=system_message,
    messages=[
        {"role": "user", "content": reasoning_prompt}
    ]
)

reasoning_message = reasoning_message.content

print(reasoning_message)


[ThinkingBlock(signature='ErUBCkYIBhgCIkAegwjkTkgzzdVwgJw6u/3c0h+TEQTOWH5EqGYH30LhAmCaFjl1GMxoe/99+Yltk1kfpQ2ovePPROLDhMwxoKb4EgxIxoZ/adgHmuE3Gs8aDP0ldWYlEOCEfPRbFCIwndHHhs2+lS846JMaoHsVjbhJEhpERuTvuYS7TxLOwu1IOPOveIWJA5i+iOHRp/ilKh31DVUXDaZ9hj6LvoMZw4f4criyr/WV9q1vkybDuxgC', thinking="Let me analyze this customer service situation:\n\n1. The customer complaint:\n   - Very brief complaint stating they were overcharged\n   - Demanding a refund\n   - No specific details provided (no order number, purchase date, product information, or amount)\n\n2. The email response being evaluated:\n   - Acknowledges the customer's frustration\n   - Requests necessary information (purchase date and order number)\n   - Promises to review the case and find a fair resolution\n   - Mentions alignment with company policy\n\n3. Company policy details:\n   - 14-day refund window\n   - Supervisor approval needed for purchases over $100\n   - Balance between customer satisfaction and preventing refund abuse\n\n

In [11]:
# reasoning_message = reasoning_message[1].text

display(Markdown(reasoning_message))

# Analysis of Customer Service Response

## Appropriateness Given Complaint and Company Policy

The response is **appropriate and well-balanced** for this situation because:

- It correctly avoids immediately promising a refund since the customer provided no details to verify eligibility against the 14-day policy
- It requests specific information (purchase date and order number) necessary to determine if the refund request falls within company guidelines
- It mentions policy alignment, setting appropriate expectations without being rigid
- It neither denies nor guarantees a refund before having all relevant information, which protects the company while keeping resolution options open

## Professional Tone and Empathy

The response demonstrates strong customer service qualities:

- Opens with an acknowledgment of the customer's frustration, showing empathy toward their concern
- Maintains a professional, non-defensive tone despite the demanding nature of the complaint
- Provides clear next steps rather than leaving the interaction open-ended
- Strikes a good balance between being helpful and maintaining company procedures
- Communicates briefly but effectively without unnecessary explanations

## Overall Assessment

This is an exemplary initial response that handles a vague complaint professionally. It shows empathy while gathering necessary information to evaluate the claim according to policy. The agent has successfully balanced customer service with proper procedure.

# **Your Challenge!!!**

A customer has submitted a complaint. Based on company policy and the situation, the model must select the correct resolution option from a list, and give a 1-line justification.

In [ ]:
complaint = "I ordered a ceramic mug and received a broken one. Please fix this!"

policy = """
- Broken or damaged items must be reported within 7 days of delivery.
- If reported in time, the item is replaced or refunded.
- No action is taken for reports after 7 days unless escalated.
"""

metadata = """
    Reported Days After Delivery: 5
    Item Value: $12
    Customer Tier: Standard
    Past Complaints: 0
"""


Build a customer support agent that could help solve this issue

The agent should choose one of the following options:
- Refund Item
- Replace Item
- Offer Discount
- Apologize (no action)
- Report to supervisor

In [ ]:
system_message = "You are a ..."

prompt = ""

message =

## **Reflections**

1. What reasoning steps did the model take to select its resolution? Were they logical and aligned with company policy? (Hint: Have a look at the reasoning output)
2. If you were designing the agent, what would you change about its decision-making process?
3. What did you learn about prompt design and reasoning models from this exercise?